# --- Day 18: Lavaduct Lagoon ---
Thanks to your efforts, the machine parts factory is one of the first factories up and running since the lavafall came back. However, to catch up with the large backlog of parts requests, the factory will also need a large supply of lava for a while; the Elves have already started creating a large lagoon nearby for this purpose.

However, they aren't sure the lagoon will be big enough; they've asked you to take a look at the dig plan (your puzzle input). For example:
```
R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)
```
The digger starts in a 1 meter cube hole in the ground. They then dig the specified number of meters up (U), down (D), left (L), or right (R), clearing full 1 meter cubes as they go. The directions are given as seen from above, so if "up" were north, then "right" would be east, and so on. Each trench is also listed with the color that the edge of the trench should be painted as an RGB hexadecimal color code.

When viewed from above, the above example dig plan would result in the following loop of trench (#) having been dug out from otherwise ground-level terrain (.):
```
#######
#.....#
###...#
..#...#
..#...#
###.###
#...#..
##..###
.#....#
.######
```
At this point, the trench could contain 38 cubic meters of lava. However, this is just the edge of the lagoon; the next step is to dig out the interior so that it is one meter deep as well:
```
#######
#######
#######
..#####
..#####
#######
#####..
#######
.######
.######
```
Now, the lagoon can contain a much more respectable 62 cubic meters of lava. While the interior is dug out, the edges are also painted according to the color codes in the dig plan.

The Elves are concerned the lagoon won't be large enough; if they follow their dig plan, how many cubic meters of lava could it hold?

## --- Part Two ---
The Elves were right to be concerned; the planned lagoon would be much too small.

After a few minutes, someone realizes what happened; someone swapped the color and instruction parameters when producing the dig plan. They don't have time to fix the bug; one of them asks if you can extract the correct instructions from the hexadecimal codes.

Each hexadecimal code is six hexadecimal digits long. The first five hexadecimal digits encode the distance in meters as a five-digit hexadecimal number. The last hexadecimal digit encodes the direction to dig: 0 means R, 1 means D, 2 means L, and 3 means U.

So, in the above example, the hexadecimal codes can be converted into the true instructions:
```
#70c710 = R 461937
#0dc571 = D 56407
#5713f0 = R 356671
#d2c081 = D 863240
#59c680 = R 367720
#411b91 = D 266681
#8ceee2 = L 577262
#caa173 = U 829975
#1b58a2 = L 112010
#caa171 = D 829975
#7807d2 = L 491645
#a77fa3 = U 686074
#015232 = L 5411
#7a21e3 = U 500254
```
Digging out this loop and its interior produces a lagoon that can hold an impressive 952408144115 cubic meters of lava.

Convert the hexadecimal color codes into the correct instructions; if the Elves follow this new dig plan, how many cubic meters of lava could the lagoon hold?

In [1]:
from pathlib import Path
import os

yr = 2023
d = 18

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
import numpy as  np

def format_input(inp):
  formatted_input = []
  for l in inp.splitlines():
    spl = l.split(' ')
    formatted_input.append((spl[0],
                            int(spl[1]),
                            spl[2].replace('(', '').replace(')', ''))
    )
  return formatted_input

In [3]:
def dig_trench(formatted_input, use_hex_instructions=False):
  import numpy as np

  if use_hex_instructions:
    directions = [{0:'R', 1:'D', 2:'L', 3:'U'}[int(x[2][-1])] for x in formatted_input]
    lengths = [int(x[2][1:-1], 16) for x in formatted_input]
  else:
    directions = [x[0] for x in formatted_input]
    lengths = [x[1] for x in formatted_input]

  assert(len(directions)==len(lengths))


    
  dir_2_delta = {'U': [-1,0], 'D': [1,0], 'L': [0,-1], 'R': [0,1]}

  max_vert = max(sum([l for d, l in zip(directions, lengths) if d=='U']), sum([l for d, l in zip(directions, lengths) if d=='D']))
  max_horiz = max(sum([l for d, l in zip(directions, lengths) if d=='L']), sum([l for d, l in zip(directions, lengths) if d=='R']))

  arr = np.full(((2*max_vert+1), (2*max_horiz+1)), '.', dtype=str)

  origin = (max_vert//2, max_horiz//2)
  arr[origin] = '#'

  cur = origin
  for d, l in zip(directions, lengths):
    for _ in range(l):
      cur = tuple(np.add(list(cur), dir_2_delta[d]))
      arr[cur] = '#'

  min_horiz_trench_loc = min([i for i in range(arr.shape[0]) if '#' in arr[i,:]])
  max_horiz_trench_loc = max([i for i in range(arr.shape[0]) if '#' in arr[i,:]])
  min_vert_trench_loc = min([i for i in range(arr.shape[1]) if '#' in arr[:,i]])
  max_vert_trench_loc = max([i for i in range(arr.shape[1]) if '#' in arr[:,i]])

  arr = arr[min_horiz_trench_loc-1:max_horiz_trench_loc+2, min_vert_trench_loc-1:max_vert_trench_loc+2]
  return arr

def fill_trench(arr):


  def to_north(l):
    return (l[0]-1, l[1])

  def to_south(l):
    return (l[0]+1, l[1])

  def to_east(l):
    return (l[0], l[1]+1)

  def to_west(l):
    return (l[0], l[1]-1)


  def is_on_map(l):
    # Is this location out of the bounds of the map
    return ((l[0]>=0 and l[0]<arr.shape[0])
            and (l[1]>=0 and l[1]<arr.shape[1]))

  def find_outside_neighbors(loc):
    return [p for p in [x for x in
      [to_north(loc), to_south(loc), to_east(loc), to_west(loc)]
                        if is_on_map(x)]
      if arr[p] in ['.']]


  # Points we've already examined
  known_outside = []
  hknown_outside = {}
  # Points that are on our current frontier of search
  queue = []
  hqueue = {}

  loc = (0,0)
  if loc not in known_outside and arr[loc] == '.':
    cur_loc = loc
    to_queue = [x for x in find_outside_neighbors(cur_loc)
    if (x not in hknown_outside and x not in hqueue)]
    queue += to_queue
    for l in to_queue:
      hqueue[l] = True
    known_outside.append(cur_loc)
    hknown_outside[cur_loc] = True
    while len(queue)!=0:
      cur_loc = queue.pop()
      del hqueue[cur_loc]
      to_queue = [x for x in find_outside_neighbors(cur_loc)
      if (x not in hknown_outside and x not in hqueue)]
      queue += to_queue
      for l in to_queue:
        hqueue[l] = True
      known_outside.append(cur_loc)
      hknown_outside[cur_loc] = True
      # print(len(known_outside))

  known_outside = set(known_outside)
  for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
      if (i,j) not in known_outside:
        arr[(i,j)] = '#'

  return arr

In [4]:
def get_trench_volume_naive(formatted_input, use_hex_instructions=False):
  '''
  Build trench and use floodfill to determine which spaces are in/out
  Then count inner spaces
  '''
  return sum(fill_trench(dig_trench(formatted_input, use_hex_instructions=use_hex_instructions)).flatten()=='#')

In [5]:
def get_trench_volume_shoelace(formatted_input, use_hex_instructions=False):
  '''
  Use Shoelace Theorem to get number of inner spaces.
  Add border spaces according to Pick's Theorem.
  '''
  if use_hex_instructions:
    directions = [{0:'R', 1:'D', 2:'L', 3:'U'}[int(x[2][-1])] for x in formatted_input]
    lengths = [int(x[2][1:-1], 16) for x in formatted_input]
  else:
    directions = [x[0] for x in formatted_input]
    lengths = [x[1] for x in formatted_input]

  assert(len(directions)==len(lengths))

  # Shoelace Theorem
  dir_2_delta = {'U': [-1,0], 'D': [1,0], 'L': [0,-1], 'R': [0,1]}
  s = np.zeros(1, dtype=np.longlong)
  cur_loc = (0,0)
  for i in range(len(lengths)):
    next_loc = tuple(np.array(dir_2_delta[directions[(i+1)%(len(lengths))]])
                    * lengths[(i+1)%(len(lengths))] + np.array(cur_loc))
    s[0] += np.subtract(np.prod([cur_loc[0],next_loc[1]], dtype=np.longlong), np.prod([cur_loc[1], next_loc[0]], dtype=np.longlong))
    cur_loc = next_loc
  s[0] /= 2

  # Pick's Theorem
  return int(abs(s[0]) + sum(lengths)/2 + 1)


In [6]:
import time

t = time.time()

formatted_input = format_input(inp)

print(get_trench_volume_naive(formatted_input, use_hex_instructions=False))
print(get_trench_volume_shoelace(formatted_input, use_hex_instructions=True))

print('\nRUNTIME: ', time.time()-t)

49061
92556825427032

RUNTIME:  0.538588285446167


In [7]:
# print('\n'.join([''.join(x) for x in fill_trench(dig_trench(formatted_input))]))